# Import Python Packages

In [2]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat



## import PyOD packages and methods 

In [4]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

## Import Metrics Packages

In [5]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define Data file and read X & Y

In [6]:
mat_file_list = ['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat',
                'musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat',
                'vertebral.mat','vowels.mat','wbc.mat']

In [7]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

## How to Load mat file

## Input (Independent) Feature shape in Mat File format

In [35]:
data = loadmat('Anamoly_detec_data/arrhythmia.mat')
type(data['X']),data['X'].shape

(numpy.ndarray, (452, 274))

## Dependent Target/Output Feature shape

In [17]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

## Define nine outlier detection tools to be compared

In [18]:
df_columns = ['Data','#Samples','# Dimensions','Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

## ROC(Region of Characteristics) Performance Evolution table

In [19]:
roc_df
#Created empty dataset which contains all columns

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


## precision_n_scores - Performance evolution table

In [21]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


## Time Dataframe

In [22]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Exploring all Mat files

In [36]:
random_state = np.random.RandomState(42)
from time import time

for mat_file in mat_file_list:
    print("\n...Processing", mat_file, '....')
    mat = loadmat(os.path.join('Anamoly_detec_data', mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits = 4)
    
    #Construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
            
    #60% Data for training & 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = random_state)
    
    # Standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {
    'ABOD': ABOD(contamination=outliers_fraction),
    'CBLOF': CBLOF(contamination=outliers_fraction, check_estimator=False, random_state=random_state),
    'FeatureBagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
    'HBOS': HBOS(contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
    'KNN': KNN(contamination=outliers_fraction),
    'LOF': LOF(contamination=outliers_fraction),
    'MCD': MCD(contamination=outliers_fraction, random_state=random_state),
    'OCSVM': OCSVM(contamination=outliers_fraction),
    'PCA': PCA(contamination=outliers_fraction, random_state=random_state),
    }
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis = 0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis = 0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis = 0)



...Processing arrhythmia.mat ....
ABOD ROC:0.7687, precision @ rank n:0.3571, execution time: 0.1177s


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


CBLOF ROC:0.7684, precision @ rank n:0.4643, execution time: 0.1072s
FeatureBagging ROC:0.7799, precision @ rank n:0.5, execution time: 0.5013s
HBOS ROC:0.8511, precision @ rank n:0.5714, execution time: 0.0539s
Isolation Forest ROC:0.8478, precision @ rank n:0.5357, execution time: 0.4481s
KNN ROC:0.782, precision @ rank n:0.5, execution time: 0.1007s
LOF ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0728s


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


MCD ROC:0.8228, precision @ rank n:0.4286, execution time: 0.9748s
OCSVM ROC:0.7986, precision @ rank n:0.5, execution time: 0.0359s
PCA ROC:0.7997, precision @ rank n:0.5, execution time: 0.0808s

...Processing cardio.mat ....
ABOD ROC:0.5763, precision @ rank n:0.1875, execution time: 0.363s
CBLOF ROC:0.8221, precision @ rank n:0.4844, execution time: 0.1216s


...

CBLOF ROC:0.3908, precision @ rank n:0.0, execution time: 0.0588s
FeatureBagging ROC:0.3027, precision @ rank n:0.0, execution time: 0.0443s
HBOS ROC:0.2695, precision @ rank n:0.0, execution time: 0.003s
Isolation Forest ROC:0.3576, precision @ rank n:0.0, execution time: 0.377s
KNN ROC:0.318, precision @ rank n:0.0, execution time: 0.013s
LOF ROC:0.318, precision @ rank n:0.0, execution time: 0.004s
MCD ROC:0.3308, precision @ rank n:0.0, execution time: 0.0632s
OCSVM ROC:0.4087, precision @ rank n:0.0, execution time: 0.0287s
PCA ROC:0.3397, precision @ rank n:0.0, execution time: 0.002s

...Processing vowels.mat ....
ABOD ROC:0.9521, precision @ rank n:0.4706, execution time: 0.37s
CBLOF ROC:0.9278, precision @ rank n:0.4118, execution time: 0.0678s


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


FeatureBagging ROC:0.9385, precision @ rank n:0.3529, execution time: 0.2962s
HBOS ROC:0.6758, precision @ rank n:0.1765, execution time: 0.005s
Isolation Forest ROC:0.7469, precision @ rank n:0.1176, execution time: 0.4857s
KNN ROC:0.9568, precision @ rank n:0.5294, execution time: 0.1037s
LOF ROC:0.9345, precision @ rank n:0.4118, execution time: 0.03s
MCD ROC:0.6779, precision @ rank n:0.0, execution time: 0.754s
OCSVM ROC:0.7415, precision @ rank n:0.2941, execution time: 0.0259s
PCA ROC:0.5787, precision @ rank n:0.1176, execution time: 0.002s

...Processing wbc.mat ....
ABOD ROC:0.9232, precision @ rank n:0.3, execution time: 0.0558s
CBLOF ROC:0.9063, precision @ rank n:0.6, execution time: 0.0409s


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


FeatureBagging ROC:0.9415, precision @ rank n:0.5, execution time: 0.0708s
HBOS ROC:0.9592, precision @ rank n:0.7, execution time: 0.006s
Isolation Forest ROC:0.9451, precision @ rank n:0.5, execution time: 0.2613s
KNN ROC:0.9437, precision @ rank n:0.5, execution time: 0.015s
LOF ROC:0.9352, precision @ rank n:0.4, execution time: 0.006s
MCD ROC:0.8986, precision @ rank n:0.4, execution time: 0.0658s
OCSVM ROC:0.9408, precision @ rank n:0.5, execution time: 0.004s
PCA ROC:0.9324, precision @ rank n:0.6, execution time: 0.002s


In [37]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971


In [38]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893


In [39]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.1626,0.1523,0.4867,0.0489,0.4019,0.0848,0.0678,0.8522,0.0369,0.0458
0,cardio,1831,21,9.6122,0.2903,0.1039,0.6942,0.0054,0.3418,0.1207,0.0778,0.4015,0.0608,0.012
0,glass,214,9,4.2056,0.0259,0.0309,0.0279,0.004,0.2726,0.008,0.003,0.0565,0.003,0.001
0,ionosphere,351,33,35.8974,0.0618,0.0459,0.0698,0.009,0.2862,0.014,0.006,0.0439,0.005,0.0037
0,letter,1600,32,6.25,0.2833,0.0877,0.6224,0.009,0.378,0.1078,0.0708,0.738,0.0658,0.004
0,lympho,148,18,4.0541,0.0209,0.0369,0.0289,0.003,0.3042,0.006,0.002,0.028,0.001,0.001
0,arrhythmia,452,274,14.6018,0.1177,0.1072,0.5013,0.0539,0.4481,0.1007,0.0728,0.9748,0.0359,0.0808
0,cardio,1831,21,9.6122,0.363,0.1216,0.6772,0.006,0.3836,0.1307,0.0818,0.4887,0.0708,0.018
0,glass,214,9,4.2056,0.0349,0.0339,0.0349,0.002,0.2573,0.008,0.002,0.028,0.001,0.002
0,ionosphere,351,33,35.8974,0.0688,0.0519,0.0708,0.006,0.2873,0.015,0.006,0.0449,0.004,0.002
